In [59]:
import os
import cv2
import pandas as pd
from tqdm import tqdm
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor, Compose
from sklearn.model_selection import train_test_split
from PIL import Image,  ImageEnhance

from sklearn.metrics import f1_score, accuracy_score, classification_report,  confusion_matrix

import mediapipe as mp

In [60]:
DATA_DIR = 'slovo'
TRAIN_DIR = os.path.join(DATA_DIR, 'train_100')
TEST_DIR = os.path.join(DATA_DIR, 'test_100')
ann_100 = pd.read_csv(os.path.join(DATA_DIR, 'annotations_100.csv'))
ann_100

,attachment_id,text,user_id,height,width,length,train,frame_count,height_width
0,5ce6ce8a-d191-47d4-97b4-3fc6f9138a73,я,185bd3a81d9d618518d10abebf0d17a8,1920,1080,91.0,True,51,1920_1080
1,c9e2dbd1-ec2f-4382-a80e-5bca8396a9b1,я,46dd04a1caa75ed3082b573cb5a3ad26,1920,1080,58.0,True,58,1920_1080
2,3f72244c-7379-4403-b7ee-e1b1b8d78d46,я,db573f94204e56e0cf3fc2ea000e5bdc,1280,720,49.0,True,49,1280_720
3,5adf08e2-4ff6-4183-ba22-446c1cd0b0f8,я,0211b488644476dd0fec656ccb9b74fc,1920,1080,50.0,True,50,1920_1080
4,13a148fc-a546-47ab-b5d0-f9d0c3cce1de,я,2d84da20c251acaeb3186642fcb04f2e,1920,1080,27.0,True,27,1920_1080
...,...,...,...,...,...,...,...,...,...
1995,74db1174-1314-43c7-bb8a-cc5cce5e2510,редко,f2dc6eab563f93d86629c1cfe479f09d,1440,1440,56.0,False,56,1440_1440
1996,5d742908-98ce-4172-bb2f-02f3595860a4,редко,798aa826b6129d33f72f62d4ba60b681,720,1280,61.0,False,61,720_1280
1997,dfe2ff4c-98fa-4ead-b861-f499f570a321,месяц,798aa826b6129d33f72f62d4ba60b681,720,1280,58.0,False,58,720_1280
1998,04856a31-8326-49cb-8e0c-0843c64a1d92,время,798aa826b6129d33f72f62d4ba60b681,720,1280,52.0,False,52,720_1280


In [64]:
# Create a Holistic object to detect pose, face, and hands keypoints
mp_holistic = mp.solutions.holistic

# Drawing utilities
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    if image is None:
        raise ValueError(f"Failed to load image: {image}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [66]:
# Path to the video file
video_path = './благодарю_cut.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
output_path = './благодарю_new.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks on the frame
        draw_styled_landmarks(image, results)

        # Write the frame with landmarks to the output video file
        out.write(image)

        # Display the frame (optional)
        cv2.imshow('Frame with Landmarks', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Инференс

In [7]:
# Инициализация MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Определение вспомогательных функций
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def adjust_landmarks(arr, center):
    arr_reshaped = arr.reshape(-1, 3)
    center_repeated = np.tile(center, (len(arr_reshaped), 1))
    arr_adjusted = arr_reshaped - center_repeated
    arr_adjusted = arr_adjusted.reshape(-1)
    return arr_adjusted

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    nose = pose[:3]
    lh_wrist = lh[:3]
    rh_wrist = rh[:3]
    pose_adjusted = adjust_landmarks(pose, nose)
    lh_adjusted = adjust_landmarks(lh, lh_wrist)
    rh_adjusted = adjust_landmarks(rh, rh_wrist)
    return pose_adjusted, lh_adjusted, rh_adjusted

def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))


In [10]:
# Определение модели и загрузка весов
class TransformerModel(nn.Module):
    def __init__(self, input_size, num_classes, num_heads=5, dim_feedforward=128, num_layers=2):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_size, input_size)  # линейный слой для преобразования размерности входных данных
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = x.permute(1, 0, 2)  # Transformer ожидает размерность (seq_len, batch, features)
        x = self.embedding(x)  # Преобразование размерности входных данных
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # Усреднение по всем временным шагам
        x = self.fc(x)
        return x

input_size = 225
num_classes = 100
model = TransformerModel(input_size, num_classes)
model.load_state_dict(torch.load('best_model_transformer_v1.pth'))
model.eval()

TransformerModel(
  (embedding): Linear(in_features=225, out_features=225, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=225, out_features=225, bias=True)
        )
        (linear1): Linear(in_features=225, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=225, bias=True)
        (norm1): LayerNorm((225,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((225,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=225, out_features=100, bias=True)
)

In [50]:
# Инференс на видео и сохранение результата
def run_inference_on_video(video_path, output_video_path, model, holistic, num_frames=20):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (height, height))

    keypoints_sequence = []

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, total_frames - 1, min(num_frames, total_frames), dtype=int)

        frame_index = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_index > frame_indices[-1]:
                break

            if frame_index in frame_indices:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                h, w, _ = frame_rgb.shape
                if h > w:
                    padding = (h - w) // 2
                    frame_rgb = cv2.copyMakeBorder(frame_rgb, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])
                else:
                    padding = (w - h) // 2
                    frame_rgb = cv2.copyMakeBorder(frame_rgb, padding, padding, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])

                results = holistic.process(frame_rgb)
                pose_adjusted, lh_adjusted, rh_adjusted = extract_keypoints(results)
                keypoints_sequence.append(np.concatenate([pose_adjusted, lh_adjusted, rh_adjusted]))

                frame_rgb = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
                draw_styled_landmarks(frame_rgb, results)
                out.write(frame_rgb)

            frame_index += 1

        cap.release()
        out.release()

    while len(keypoints_sequence) < num_frames:
        keypoints_sequence.append(np.zeros(33*3 + 21*3 + 21*3))

    keypoints_sequence = np.array(keypoints_sequence)
    keypoints_sequence = torch.tensor(keypoints_sequence, dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        predictions = model(keypoints_sequence)
        predicted_class = torch.argmax(predictions, dim=1).item()

    return predicted_class


In [51]:
class_data2 = pd.read_csv(os.path.join(DATA_DIR, 'class_data_3.csv'))
class_data2 = class_data2.set_index('label')['class_name'].to_dict()

In [52]:
# Пример использования
video_path = './день_cut.mp4'
output_video_path = './день_cut_inference.mp4'
predicted_class = run_inference_on_video(video_path, output_video_path, model, mp_holistic)
print(f'Predicted Class: {class_data2[predicted_class]}')

Predicted Class: день


In [55]:
video_path = './благодарю_cut.mp4'
output_video_path = './благодарю_cut_inference.mp4'
predicted_class = run_inference_on_video(video_path, output_video_path, model, mp_holistic)
print(f'Predicted Class: {class_data2[predicted_class]}')

Predicted Class: благодарю
